# Trabalho #1 - MC906
### Integrantes do grupo
164213 Andreza Aparecida dos Santos

166301 Daniela Palumbo

160160 Guilherme Furlan

172655 Lucas Cunha

187506 Thamiris Coelho


In [ ]:
import sys
import random
import math
import matplotlib.pyplot as plt
PATH_TO_AIMA = "../aima-python"
if PATH_TO_AIMA not in sys.path:
    sys.path.insert(0,PATH_TO_AIMA)
from search import *
import timeit
import walls

In [ ]:
# Global maze declaration
maze_array = None
time_measure = {
    "BFS": [],
    "DLS": [],
    "ASL": [],
    "ASM": [],
    "GL": [],
    "GM": [],
    "HC": [],
    "pacman_initials": [],
    "goal_positions": [],
}

## Definição do Objetivo do Trabalho

O  Pacman  encontra-se  em  um  labirinto  composto  pelas paredes, 3 fantasmas estáticos em posições pré-definidas, um ponto objetivo e dots. O objetivo desse trabalho é levar o Pacman até o ponto objetivo.
O Pacman  se  move  de  acordo  com  vizinhança-4,  a  cada  ação executada  ele  coleta  um  dot  do  labirinto  e  revisitar  posições não é permitido. 

O labirinto é representado como uma grid de tamanho variável. As paredes do labirinto são intransponíveis e o ambiente é completamente observável <span style="color:red">(talvez seja parcialmente observável)</span>, determinıstico, sequencial, estático e discreto.

## Problem Modeling

Para modelar o problema utilizamos a classe "Problem" presente na biblioteca AIMA.search[1] em Python 3.

Definimos as seguintes propriedades do problema:
- O labirinto é global
- Os estados são uma tupla (x,y,z) de modo que x e y representam a posição do Pacman no labirinto e z a quantidade de dots comidos.
- As possíveis ações são tuplas (x,y) nas direções Oeste, Norte, Sul e Leste, que são testadas nessa ordem.
- A posição inicial do Pacman é gerada aleatoriamente dentro das possíveis posições¹.
- A posição do objetivo é gerada aleatoriamente dentro das possíveis posições¹.
- As posições dos fantasmas são geradas aleatoriamente dentro das possíveis posições¹.

¹ posições possíveis são posições representadas por dots na geração do labirinto.

A modelagem foi feita definindo as funções abaixo.
- actions: Determina a partir da posição (x,y) do Pacman e do estado do labirinto quais ações são possíveis de serem executadas.
- result: Retorna o novo estado do Pacman após executar uma ação.
- goaltest: Testa se o Pacman chegou no objetivo.
- pathcost: Determina o custo para cada ação executada. Para o nosso caso, cada passo dado pelo Pacman tem custo 1.
- value: Retorna quantos dots foram coletados. É utilizada no algoritmo de busca local para ser maximizada.



In [ ]:
class PacmanMazeProblem(Problem):
    """The Pacman Maze Problem."""
    
    def __init__(self, pac_position, goal_position):
        Problem.__init__(self, pac_position, goal_position)
        
    def actions(self, state):
        """The possible actions include the neighbor positions of a given node
        except if the position is a ghost or a wall"""        
        actions = []
        y, x, _ = state
        directions = [         (0, -1),
                      (-1, 0),           (1,  0),
                                (0, 1)           ]
        for position in directions:
            possible_pos = [y+position[1], x+position[0]]
            if possible_pos[1] < 0:
                possible_pos[1] = maze_array.width-1
            elif possible_pos[1] == maze_array.width:
                possible_pos[1] = 0
            possible_pos = tuple(possible_pos)
            maze_value = maze_array.maze[possible_pos]
            if maze_value == CellType.DOT:
                actions.append(possible_pos)
        return actions

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state."""
        global maze_array
        
        maze_array.maze[state[0]][state[1]] = CellType.EMPTY
        _, _, dots = state
        state = (action[0], action[1], dots+1)
        maze_array.maze[state[0]][state[1]] = CellType.EMPTY
        
        return state
    
    def goal_test(self, state):
        """Return True if the state is the goal."""
        y, x, _ = state
        
        return y == self.goal[0] and x == self.goal[1]

    def path_cost(self, c, state1, action, state2):
        """
        Every action is a step of cost 1
        """
        return c + 1
    
    def value(self, state):
        """For optimization problems, each state has a value. Hill Climbing
        and related algorithms try to maximize this value."""
        _, _, dots = state
        return dots

### Classe Maze

A classe Maze é responsável por criar o labirinto.

Dado um tamanho de largura e altura, uma grid é criada com uma configuração aleatória de paredes e dots. Dentre as posições dos dots 5 são escolhidas aleatoriamente para representarem as posições dos 3 fantasmas, a posição inicial do Pacman e a posição do objetivo.

O código de geração do labirinto "wall.py" foi retirado do github pacman-mazegen [2].

Para entender o que representam as cores presentes nos labirintos mostradas abaixo, deve-se observar que:
    - verde: paredes intransponíveis do labirinto.
    - rosa: posições estáticas dos 3 fantasmas.
    - amarelo: posição inicial do Pacman.
    - laranja: posição do ponto objetivo.
    - vermelho: caminho final encontrado pela busca. Para a busca local, representa o ponto final de parada do Pacman.
    - branco: dots não coletados pelo Pacman.
    - preto: posições por onde o pacman passou no processo de busca pelo objetivo.

In [ ]:
class CellType:
    EMPTY = 0
    WALL = 1
    DOT = 3
    GHOST = 2

class Maze:
    def __init__(self, width, height):
        self.width = width
        self.height = height
    
    def __getitem__(self, position):
        return self.maze[position]
    
    def __setitem__(self, new_value, position):
        self.maze[position] = new_value
    
    def show_maze(self, solution):
        for point in solution:
            plt.scatter(point[1], point[0], color='red')

        plt.imshow(self.maze, interpolation='nearest', cmap=plt.cm.get_cmap("cubehelix", 4))
        plt.scatter(self.pac_position[1], self.pac_position[0], color='yellow')
        plt.scatter(self.goal_position[1], self.goal_position[0], color='orange')
        plt.axis('off')
        plt.show()
              
    def maze_to_array(self, maze_str, width, height):
        """
        Function that converts string maze to numpy array
        """
        maze = []
        for line in str(maze_str).splitlines():
            line = line+line[::-1]
            if line:
                line = list(line)
                for item in range(len(line)):
                    if line[item] == "|":
                        line[item] = CellType.WALL
                    else:
                        line[item] = CellType.DOT
                maze.append(line)
        return np.array(maze) 
    
    def maze_generator(self):
        """
        Function that generates a random maze
        Input: height and width of the maze
        Output: array maze
        """
        half_width = self.width//2
        maze = np.zeros((self.height, half_width))
        maze[0,:] = 1
        maze[:,0] = 1
        maze[-1,:] = 1
        maze[self.height//2-2:self.height//2+2, half_width-3::] = 1
        
        gap_number = random.randint(0, 3)
        for i in range(gap_number):
            maze[(i+1)*self.height//(gap_number+1), 0] = 0
        
        maze_str = []
        for line in maze:
            for item in line:
                if item == 1:
                    maze_str.append('|')
                else:
                    maze_str.append('.')
            maze_str.append('\n')
        maze_str = ''.join(maze_str)

        maze = walls.Map(half_width, self.height, maze_str)
        # generate map by adding walls until there's no more room
        while maze.add_wall_obstacle(extend=True):
            pass
        
        maze_array = self.maze_to_array(maze, half_width, self.height)
        
        # insert ghosts to the in random positions of the maze
        for i in range(3):
            valid_positions = np.argwhere(maze_array==CellType.DOT)
            position = random.choice(valid_positions)
            maze_array[position[0],position[1]] = CellType.GHOST
            print("Ghost #%d position:"%i, tuple(position))
        
        # pacman initial position is a tuple of 3 values: (x, y, dots)
        # where dots are the total dots eaten by pacman
        valid_positions = np.argwhere(maze_array==CellType.DOT)
        position = random.choice(valid_positions)
        position = np.append(position, [0])
        self.pac_position = tuple(position)
        self.goal_position = tuple(random.choice(valid_positions))
        print("Pacman initial positon: ", self.pac_position)
        print("Goal position: ", self.goal_position)
        
        # Saves the generated maze configuration
        self.maze = maze_array
        y,x, _ = self.pac_position
        self.maze[(y,x)] = CellType.EMPTY
        unique, counts = np.unique(self.maze, return_counts="True")
        print("Total number of dots in the maze: ", dict(zip(unique, counts))[CellType.DOT])
        
        # Keeps a copy of initial configuration of maze
        # Needed because the searchers change the maze
        self.original_maze = self.maze.copy()


### Heurísticas

As heurísticas são utilizadas nas funções de avaliação dos algoritmos de busca informada (A* e Best First Search, por exemplo). Para que o algoritmo encontre a solução ótima do problema, ele precisa utilizar uma heurística admissível. Uma heurística é considerada admissível quando nunca superestima o custo de de alcançar o objetivo, ou seja quando o custo para chegar do estado atual ao final nunca é maior que o menor possível.

Foram definidas duas heurísticas para o problema do PAC-MAN:
- distância de manhatan: $|(X_{1} - X_{2})| + |(Y_{1} - Y_{2})|$
- distância linear: $\sqrt{(X_{1} - X_{2})^{2} + (Y_{1} - Y_{2})^{2}}$

No caso da distância de Manhatan ela é admissível no problema do PAC-MAN, visto que, conforme nossa modelagem do problema, o pac-man só pode mover-se uma posição do labirinto por vez em somente uma das quatro direções (para cima, para baixo, esquerda e direita). A cada nova posição do pacman, o menor caminho até o objetivo final terá sempre como limite inferior a menor das distância manhattan das posições vizinhas, fazendo com que as demais posições sejam sub-ótimas, superestimando o custo final. Desta forma: 0 ≤ h(N) ≤ h*(N).

A distância Euclidiana também é admissível por ser a menor distância entre dois pontos, sendo sempre menor que o custo real para o pac-man chegar ao objetivo, já que conforme nossa modelagem ele move-se apenas na vertical e horizontal. A distância Euclidiana também é nesse caso o limite inferior do caminho que o pacman realmente terá que percorrer até o objetivo, ou seja: 0 ≤ h(N) ≤ h*(N)


In [ ]:
def linear(node):  
    Y, X, _ = node.state
    (gY, gX) = maze_array.goal_position
        
    return int(math.sqrt((gY-Y)**2+(gX-X)**2))

def manhatan(node):  
    Y, X, _ = node.state
    (gY, gX) = maze_array.goal_position
        
    return (abs(gY-Y)+abs(gX-X))

### Geração do Maze 

Gera o maze com os tamanhos de altura e largura definidos em MAZE_HEIGHT e MAZE_WIDTH.

In [ ]:
# Set the size of the Maze
MAZE_WIDTH = 50
MAZE_HEIGHT = 51

global maze_array
maze_array = Maze(MAZE_WIDTH, MAZE_HEIGHT)
maze_array.maze_generator()

In [ ]:
time_measure["pacman_initials"].append((maze_array.pac_position[0], maze_array.pac_position[1]))
time_measure["goal_positions"].append(maze_array.goal_position)

In [ ]:
problem = PacmanMazeProblem(maze_array.pac_position, maze_array.goal_position)

## Buscas

Os algoritmos de busca utilizados nesse trabalho serão rapidamente explicados. 
Para ajudar na explicação, a figura abaixo possui um exemplo de grafo com o estado inicial e o objetivo.


![alt text](./output/graph_example.png)

### Métodos de Busca Não-Informada

#### Busca em Largura 

A busca em largura realiza a visitação dos nós de modo que para cada nó pai P, todos os seus nós "filhos" são visitados antes de visitar um nó "neto".

Esse algoritmo de busca utiliza uma fila FIFO, é completo pois sempre encontra uma solução se ela existir, e a solução encontrada é ótima, pois todos os caminhos possuem o mesmo custo.

Aplicando esse algoritmo no exemplo de grafo mostrado anteriormente, a sequência de nós visitados pela busca até encontrar o objetivo (GOAL) é ABCDEFGHIJKLM.

Esse algoritmo possui:
    - complexidade de tempo O(b^(d+1)), onde b é o número médio de sucessores de cada nó e d é a profundidade da solução encontrada. Essa solução é sempre a solução de menor profundidade existente.
    - complexidade de memória O(b^(d+1)), pois todos os nós visitados são mantidos em memória de modo 
    a recuperar o caminho da solução encontrada.


#### Busca Limitada em Profundidade

A Busca Limitada em Profundidade realiza a visitação dos nós igual a Busca em Profundidade, isto é, expandindo sempre os nós mais profundos. Esse algoritmo utiliza uma pilha LIFO, escolhendo como sucessor o primeiro nó na pilha, desse modo nós inseridos por último na pilha são expandido primeiro.

A diferença entre essa busca e uma busca em profundidade padrão é que ela só expande até uma profundidade limite L. Essa busca é considerada incompleta, ou seja, não encontra uma solução caso L < d, e não é ótima para L > d, onde d é a profundidade da solução encontrada.

Aplicando esse algoritmo no exemplo de grafo mostrado anteriormente, a sequência de nós visitados pela busca até encontrar o objetivo (GOAL) é ABDHIEJKCFLM.

Esse algoritmo possui:
    - complexidade de tempo O(b^L), onde b é o número médio de 
    sucessores de cada nó.
    - complexidade de memória O(bL).

Para efeitos do trabalho L = 50, dessa forma se a distância entre o Pacman e o ponto objetivo for maior, o algoritmo não encontra uma solução.

In [ ]:
# global maze_array
maze_array.maze = maze_array.original_maze.copy()
solution = breadth_first_graph_search(problem)
print("Path Solution Found: ", solution.solution())
print("Cost of the path found: ", solution.path_cost)
print("\nFinal maze:")
maze_array.show_maze(solution.solution())

In [ ]:
global maze_array
maze_array.maze = maze_array.original_maze.copy()
solution = depth_limited_search(problem)
try:
    print("Path Solution Found: ", solution.solution())
    print("Cost of the path found: ", solution.path_cost)
    print("\nFinal maze:")
    maze_array.show_maze(solution.solution())
except:
    print("Path not found with depth limited = 50")

In [ ]:
%%timeit -o
######### Breadth First Search #########

global maze_array
maze_array.maze = maze_array.original_maze.copy()
solution = breadth_first_graph_search(problem)

In [ ]:
time_measure["BFS"].append(_.average)

In [ ]:
%%timeit -o
######### Depth Limited Search #########
global maze_array
maze_array.maze = maze_array.original_maze.copy()
solution = depth_limited_search(problem)

In [ ]:
time_measure["DLS"].append(_.average)

### Métodos de Busca Informada

#### Greedy Best First

Como um dos métodos de busca informada, aplicamos o Greedy Best First. Este algoritmo utiliza uma função heurística F(n) = h(n) como uma estimativa do custo do caminho do nó atual até o estado final, expandindo para aquele que possui uma avaliação aparentemente melhor em relação ao objetivo. No caso do labirinto do Pacman, o nó vizinho que apresenta o menor valor da função h(n) é o escolhido para ser expandido. É um algoritmo guloso por sempre realizar localmente a melhor escolha. Como exemplo, imagine que um problema cujo objetivo seja chegar ao nó Y, sendo o nó X o estado inicial, cujos vizinhos são A, B e C, possuindo os seguintes valores para a função heurística h(n):

![alt text](./output/heuristic_example.jpeg)

O nó a ser expandido seria o B, pelo valor de h(n) ser o menor dentre todos os vizinhos de X.

Este algoritmo possui:

    - Complexidade de tempo e espaço O(bm) no pior caso, porém boas heurísticas conseguem reduzir essa complexidade.
    - Pode ser incompleto se na implementação não identificar estados repitidos.

#### A*

O algoritmo A* é uma combinação da estimativa de uma heurística h(n) e da função custo para ir do nó atual até o vizinho (g(n)), resultando na seguinte função de avaliação: f(n) = g(n) + h(n). Da mesma forma que no greedy best first, o nó a ser expandido é sempre o que apresenta a melhor avaliação em relação ao objetivo, no caso do Pacman, o menor valor. Conforme modelamos o problema, o custo de g(n) sempre será unitário para todos nós vizinhos, fazendo que na prática a heurística h(n) seja a única responsável pelo resultado da função avaliação.

Este algoritmo possui:

    - Complexidade de tempo e espaço O(b^d) no pior caso, porém o uso de boas heurísticas reduz essa complexidade.
    - Pode ser ótimo e completo com a utilização de heurísticas consistentes.


In [ ]:
global maze_array
maze_array.maze = maze_array.original_maze.copy()
solution = best_first_graph_search(problem, linear)

print("Path Solution Found: ", solution.solution())
print("Cost of the path found: ", solution.path_cost)
print("\nFinal maze:")
maze_array.show_maze(solution.solution())

In [ ]:
global maze_array
maze_array.maze = maze_array.original_maze.copy()
solution = best_first_graph_search(problem, manhatan)

print("Path Solution Found: ", solution.solution())
print("Cost of the path found: ", solution.path_cost)
print("\nFinal maze:")
maze_array.show_maze(solution.solution())

In [ ]:
global maze_array
maze_array.maze = maze_array.original_maze.copy()
solution = astar_search(problem, linear)

print("Path Solution Found: ", solution.solution())
print("Cost of the path found: ", solution.path_cost)
print("\nFinal maze:")
maze_array.show_maze(solution.solution())

In [ ]:
global maze_array
maze_array.maze = maze_array.original_maze.copy()
solution = astar_search(problem, manhatan)

print("Path Solution Found: ", solution.solution())
print("Cost of the path found: ", solution.path_cost)
print("\nFinal maze:")
maze_array.show_maze(solution.solution())

In [ ]:
%%timeit -o
######### Greedy Best First with f() = linear distance #########

global maze_array
maze_array.maze = maze_array.original_maze.copy()
solution = best_first_graph_search(problem, linear)

In [ ]:
time_measure["GL"].append(_.average)

In [ ]:
%%timeit -o
######### Greedy Best First with f() = manhatan distance #########

global maze_array
maze_array.maze = maze_array.original_maze.copy()
solution = best_first_graph_search(problem, manhatan)

In [ ]:
time_measure["GM"].append(_.average)

In [ ]:
%%timeit -o
######### A* with h() = linear distance #########

global maze_array
maze_array.maze = maze_array.original_maze.copy()
solution = astar_search(problem, linear)

In [ ]:
time_measure["ASL"].append(_.average)

In [ ]:
%%timeit -o
######### A* with h() = manhatan distance #########
global maze_array
maze_array.maze = maze_array.original_maze.copy()
solution = astar_search(problem, manhatan)

In [ ]:
time_measure["ASM"].append(_.average)

### Método de Busca Local

#### Hill Climbing

Para o algoritmo de busca local escolhemos o Hill Climbing. Esse algoritmo funciona como um loop que visa continuamente aumentar a função value() definida na nossa classe "PacmanMazeProblem".

Definimos nossa função value como o número de dots coletados pelo Pacman, de modo que o Hill Climbing agisse de forma a maximizar esse valor. Vale pontuar que esse é o único objetivo desse algoritmo, ou seja, através dele o Pacman não chegará ao ponto objetivo que foi definido anteriormente e utilizados nas buscas anteriores.

O Hill Climbing não é completo, nem ótimo. Uma limitação que podemos pontuar desse algoritmo é que ele pode ficar preso em máximos locais, uma vez que não possui nenhum mecanismo que possubilite voltar atrás alguns passos para encontrar uma saída melhor.

Esse algoritmo possui:
    - Complexidade de tempo no melhor caso de O(d) e no pior caso de O(b^d), onde b é o número médio de sucessores de um nó e d a profundidade da solução encontrada.
    - Como complexidade de memória, esse algoritmo não precisa manter nós na memória.

In [ ]:
maze_array.maze = maze_array.original_maze.copy()
solution = hill_climbing(problem)
print("Number of eaten dots: ", solution[2])
maze_array.show_maze([(solution[0], solution[1])])

In [ ]:
%%timeit -o
######### Hill Climbing Local Search #########
global maze_array
maze_array.maze = maze_array.original_maze.copy()
solution = hill_climbing(problem)

In [ ]:
time_measure["HC"].append(_.average)

In [ ]:
# def compare_searchers(problems, header,
#                       searchers=[breadth_first_graph_search,
#                                  depth_limited_search,
#                                  [best_first_graph_search, 'manhatan'],
#                                  [best_first_graph_search, 'linear'],
#                                  [astar_search, 'manhatan'],
#                                  [astar_search, 'linear'],
#                                  hill_climbing]):
    
#     def do(searcher, problem):
#         p = InstrumentedProblem(problem)
#         if searcher == [astar_search, 'manhatan']:
#             searcher[0](p, manhatan)
#         elif searcher == [astar_search, 'linear']:
#             searcher[0](p, linear)
#         elif searcher == [best_first_graph_search, 'manhatan']:
#             searcher[0](p, manhatan)
#         elif searcher == [best_first_graph_search, 'linear']:
#             searcher[0](p, linear)
#         else:
#             searcher(p)
#         return p

#     table = [[name(s)] + [do(s, p) for p in problems] for s in searchers]
#     print_table(table, header)

In [ ]:
# headers = ["search algorithms", "1"]
# compare_searchers([problem], headers)

## Análises de Tempo

Para analisar o tempo de execução de cada algoritmo de busca realizamos 14 experimentos, em cada um deles variamos toda configuração do labirinto, mantendo apenas o tamanho do grid. Coletamos a média do tempo de execução em cada experimento e plotamos em função da distância de manhatan entre a posição inicial do Pacman e o objetivo. 

![alt text](./output/search_times.png)

<span style="color:red">TODO: Fazer uma análise do gráfico</span>

## Referências

[1] Aima-python: search. Disponível online em https://github.com/aimacode/aima-python/blob/master/search.py

[2] pacman-mazegen: walls. Disponível online em https://github.com/shaunlebron/pacman-mazegen/blob/gh-pages/randomfill/walls.py


<span style="color:red">TODO: Arrumar referências e citações</span>